<a href="https://colab.research.google.com/github/arteagac/xlogit/blob/master/examples/multinomial_model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Install package

In [1]:
!pip install xlogit

## Read data using pandas

In [12]:
import pandas as pd
data_file = "https://raw.githubusercontent.com/arteagac/xlogit/master/examples/data/fishing_long.csv"
df = pd.read_csv(data_file)
df

,choice_id,alternative,choice,income,price,catch
0,1,beach,0,7083.33170,157.930,0.0678
1,1,boat,0,7083.33170,157.930,0.2601
2,1,charter,1,7083.33170,182.930,0.5391
3,1,pier,0,7083.33170,157.930,0.0503
4,2,beach,0,1249.99980,15.114,0.1049
...,...,...,...,...,...,...
4723,1181,pier,0,416.66668,36.636,0.4522
4724,1182,beach,0,6250.00130,339.890,0.2537
4725,1182,boat,1,6250.00130,235.436,0.6817
4726,1182,charter,0,6250.00130,260.436,2.3014


## Import Choice Model

In [0]:
from xlogit import ChoiceModel

## Estimate model with both alternative specific and individual specific variables

In [10]:
varnames = ['income','price']
X = df[varnames].values
y = df['choice'].values

model = ChoiceModel()
model.fit(X,y,isvars = ['income'], asvars=['price'],alternatives=['beach','boat','charter','pier'],varnames= varnames)
model.summary()

Optimization succesfully completed after 11 iterations. Use .summary() to see the estimated values
-----------------------------------------------------------------------------------------
Coefficient          	Estimate 	Std. Error 	z-value 	Pr(>|z|)     
-----------------------------------------------------------------------------------------
_intercept.boat  	0.4928935957 	0.2053370982 	2.4004118111 	0.0449401617 .    
_intercept.charter 	1.8540668405 	0.2097451458 	8.8396173995 	0.0000000000 ***  
_intercept.pier  	0.7526662342 	0.2042533633 	3.6849637242 	0.0009279897 **   
income.boat      	0.0000933295 	0.0000471101 	1.9810953073 	0.1122778277      
income.charter   	-0.0000324867 	0.0000478462 	-0.6789828929 	0.6333945307      
income.pier      	-0.0001267191 	0.0000465724 	-2.7209078124 	0.0198559789 .    
price            	-0.0255642838 	0.0015153615 	-16.8700891463 	0.0000000000 ***  
-----------------------------------------------------------------------------------------
Si

## Estimate model with only individual specific variables


In [14]:
varnames = ['income']
X = df[varnames].values
y = df['choice'].values

model = ChoiceModel()
model.fit(X,y,isvars=['income'],alternatives=['beach','boat','charter','pier'],varnames= varnames)
model.summary()

Optimization succesfully completed after 7 iterations. Use .summary() to see the estimated values
-----------------------------------------------------------------------------------------
Coefficient          	Estimate 	Std. Error 	z-value 	Pr(>|z|)     
-----------------------------------------------------------------------------------------
_intercept.boat  	0.7389145443 	0.1946426257 	3.7962627224 	0.0006147513 **   
_intercept.charter 	1.3412894454 	0.1909360702 	7.0248091101 	0.0000000000 **   
_intercept.pier  	0.8141524803 	0.2246575748 	3.6239707520 	0.0011570320 *    
income.boat      	0.0000919062 	0.0000393881 	2.3333488935 	0.0526384330      
income.charter   	-0.0000316409 	0.0000403088 	-0.7849622928 	0.5860992916      
income.pier      	-0.0001434047 	0.0000519035 	-2.7629112591 	0.0177071168 .    
-----------------------------------------------------------------------------------------
Significance:  *** 0    ** 0.001    * 0.01    . 0.05

Log-Likelihood= -1477.151
